# Australian Open Predicition

## Data Loading

In [41]:
# Import libraries
import os
import numpy as np
import pandas as pd

In [42]:
# Get current working directory
src =  os.getcwd()
# Load data for male players
maleDF = pd.read_csv(src+'\\data\\ATP_matches.csv')
# Load data for female players
femaleDF = pd.read_csv(src+'\\data\\WTA_matches.csv')

# Display tensor sizes
print("Male data: \n" + str(maleDF.shape[0]) + " Observations of " + str(maleDF.shape[1]) + " Variables\n")
print("Female Data: \n" + str(femaleDF.shape[0]) + " Observations of " + str(femaleDF.shape[1]) + " Variables")

Male data: 
20807 Observations of 35 Variables

Female Data: 
13906 Observations of 35 Variables


c:\users\jasper\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,13,14,17,18,24,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
# Show top of male dataframe
maleDF.head()

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,2-Jan-12,Hard,First Round,106,97,0,2,...,3,21,33,13,26,1,3,15,49,49
1,Dudi Sela,Fabio Fognini,Chennai,2-Jan-12,Hard,First Round,83,48,0,2,...,4,17,32,5,26,0,1,8,33,30
2,Go Soeda,Frederico Gil,Chennai,2-Jan-12,Hard,First Round,120,102,0,2,...,2,45,70,18,35,2,4,36,103,99
3,Yuki Bhambri,Karol Beck,Chennai,2-Jan-12,Hard,First Round,345,101,0,2,...,1,15,33,13,29,2,3,15,46,43
4,Yuichi Sugita,Olivier Rochus,Chennai,2-Jan-12,Hard,First Round,235,67,0,2,...,0,19,32,13,22,1,7,30,78,62


In [44]:
# Show top of female dataframe
femaleDF.head()

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Johanna Larsson,Lourdes Dominguez-Lino,Auckland,1-Jan-14,Hard,First Round,84,69,0.0,2,...,4,16,32,13,33,2,8,24,62,53
1,Ana Konjuh,Roberta Vinci,Auckland,1-Jan-14,Hard,First Round,259,14,0.0,2,...,1,34,65,10,23,5,6,30,68,74
2,Ayumi Morita,Lucie Safarova,Auckland,1-Jan-14,Hard,First Round,60,29,0.0,2,...,3,21,32,12,32,3,5,26,68,59
3,Venus Williams,Andrea Hlavackova,Auckland,1-Jan-14,Hard,First Round,47,134,0.0,2,...,4,23,34,12,28,2,6,30,82,65
4,Julia Goerges,Karin Knapp,Auckland,1-Jan-14,Hard,First Round,72,41,0.0,2,...,4,52,70,30,58,2,2,28,102,110


In [89]:
maleData = maleDF.head(24) 
maleWinner = maleData.groupby(['Winner', 'Tournament'])
maleLoser = maleData.groupby(['Loser', 'Tournament'])
print(maleWinner.get_group(('David Goffin', 'Chennai')).values)
print(maleLoser.get_group(('David Goffin', 'Chennai')).values)
maleWinner['Winner_FirstServes_Won', 'Winner_SecondServes_Won'].agg({'Winner_FirstServes_Won':'sum', 'Winner_SecondServes_Won':'sum'})
maleLoser['Loser_FirstServes_Won', 'Loser_SecondServes_Won'].agg({'Loser_FirstServes_Won' : 'sum', 'Loser_SecondServes_Won':'sum'})

##testData.groupby('Winner').groups

[['David Goffin' 'Xavier Malisse' 'Chennai' '2-Jan-12' 'Hard'
  'First Round' '174' '49' 0 '2' 16 '8' '3' 40 51 '18' '34' '3' '12' '27'
  '77' 85 '1' 11 '4' '1' 31 43 '19' '34' '1' '4' '27' '85' 77]
 ['David Goffin' 'Andreas Beck' 'Chennai' '2-Jan-12' 'Hard'
  'Second Round' '174' '98' 0 '2' 16 '6' '4' 30 43 '25' '39' '4' '16'
  '41' '98' 96 '1' 12 '4' '0' 43 71 '14' '27' '2' '8' '27' '82' 84]]
[['Janko Tipsarevic' 'David Goffin' 'Chennai' '2-Jan-12' 'Hard'
  'Quarter-finals' '9' '174' 0 '2' 15 '7' '0' 36 44 '13' '23' '4' '13'
  '38' '91' 87 '1' 10 '10' '5' 34 51 '19' '40' '1' '2' '18' '67' 71]]


,,Loser_FirstServes_Won,Loser_SecondServes_Won
Loser,Tournament,,
Andreas Beck,Chennai,43,14
Benoit Paire,Chennai,40,21
David Goffin,Chennai,34,19
Dudi Sela,Chennai,23,19
Edouard Roger-Vasselin,Chennai,43,16
Eric Prodon,Chennai,21,13
Fabio Fognini,Chennai,17,5
Frederico Gil,Chennai,45,18
Go Soeda,Chennai,21,10


In [49]:
maleData

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,2-Jan-12,Hard,First Round,106,97,0,2,...,3,21,33,13,26,1,3,15,49,49
1,Dudi Sela,Fabio Fognini,Chennai,2-Jan-12,Hard,First Round,83,48,0,2,...,4,17,32,5,26,0,1,8,33,30
2,Go Soeda,Frederico Gil,Chennai,2-Jan-12,Hard,First Round,120,102,0,2,...,2,45,70,18,35,2,4,36,103,99
3,Yuki Bhambri,Karol Beck,Chennai,2-Jan-12,Hard,First Round,345,101,0,2,...,1,15,33,13,29,2,3,15,46,43
4,Yuichi Sugita,Olivier Rochus,Chennai,2-Jan-12,Hard,First Round,235,67,0,2,...,0,19,32,13,22,1,7,30,78,62
5,Benoit Paire,Pere Riba,Chennai,2-Jan-12,Hard,First Round,95,89,0,2,...,5,13,20,12,32,0,1,9,44,34
6,Victor Hanescu,Sam Querrey,Chennai,2-Jan-12,Hard,First Round,90,93,0,2,...,8,29,41,7,24,1,3,17,57,53
7,Yen-Hsun Lu,Thiemo de Bakker,Chennai,2-Jan-12,Hard,First Round,82,223,0,2,...,0,20,32,10,24,1,1,19,57,49
8,Andreas Beck,Vasek Pospisil,Chennai,2-Jan-12,Hard,First Round,98,119,0,2,...,3,39,57,16,38,1,5,24,74,79
9,Ivan Dodig,Vishnu Vardhan,Chennai,2-Jan-12,Hard,First Round,36,313,0,2,...,5,41,59,13,27,2,8,34,101,88
